In [1]:
import os
os.environ["LANGCHAIN_TRACING"] = "true" # If you want to trace the execution of the program, set to "true"

from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, load_tools

from langchain.agents.agent_toolkits import PlayWrightBrowserToolkit
from langchain.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser, # A synchronous browser is available, though it isn't compatible with jupyter.
)

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()

In [2]:
def get_input() -> str:
  """Placeholder for Slack input from user."""
  print("Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.")
  contents = []
  while True:
    try:
      line = input()
    except EOFError:
      break
    if line == "q":
      break
    contents.append(line)
  return "\n".join(contents)


In [3]:
# MEMORY 
from langchain.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory

chat_history = MessagesPlaceholder(variable_name="chat_history")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [4]:
# LLM 
llm = ChatOpenAI(temperature=0, model="gpt-4-0613", max_retries=3, request_timeout=60 * 3)

In [5]:
# TOOLS
async_browser = create_async_playwright_browser()
browser_toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = browser_toolkit.get_tools()
tools

human_tools = load_tools(
    ["human"],
    llm=llm,
    input_func=get_input
)

In [6]:
tools = tools + human_tools
tools

[ClickTool(name='click_element', description='Click on an element with the given CSS selector', args_schema=<class 'langchain.tools.playwright.click.ClickToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/kastanday/Library/Caches/ms-playwright/chromium-1055/chrome-mac/Chromium.app/Contents/MacOS/Chromium> version=112.0.5615.29>, visible_only=True, playwright_strict=False, playwright_timeout=1000),
 NavigateTool(name='navigate_browser', description='Navigate a browser to the specified URL', args_schema=<class 'langchain.tools.playwright.navigate.NavigateToolInput'>, return_direct=False, verbose=False, callbacks=None, callback_manager=None, tags=None, metadata=None, handle_tool_error=False, sync_browser=None, async_browser=<Browser type=<BrowserType name=chromium executable_path=/Users/kastanday/Library/Ca

In [7]:
agent_chain = initialize_agent(
  tools, 
  llm, 
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION, 
  verbose=True,
  memory=memory, 
  agent_kwargs = {
    "memory_prompts": [chat_history],
    "input_variables": ["input", "agent_scratchpad", "chat_history"]
  }
)

In [8]:
# asycn and sync together is tough...

response = await agent_chain.arun(input="Browse to https://lumetta.web.engr.illinois.edu/120-binary/120-binary.html and complete the first challenge using the keyboard to enter information, please.")
print(response)



> Entering new AgentExecutor chain...


Thought: The assistant cannot physically interact with a keyboard or a webpage. It can only perform actions such as navigating to a URL, clicking on elements, extracting text, and so on. I should inform the user about this limitation.
Action:
```
{
  "action": "Final Answer",
  "action_input": "I'm sorry, but as an AI, I don't have the ability to physically interact with a keyboard or a webpage. I can perform actions such as navigating to a URL, clicking on elements, extracting text, and so on. However, I can guide you through the process if you'd like."
}
```

> Finished chain.
I'm sorry, but as an AI, I don't have the ability to physically interact with a keyboard or a webpage. I can perform actions such as navigating to a URL, clicking on elements, extracting text, and so on. However, I can guide you through the process if you'd like.


In [16]:
# asycn and sync together is tough...

response = await agent_chain.arun(input="Ask the user what they're interested in learning about on Langchain, then Browse to blog.langchain.dev and summarize the text especially whatever is relevant to the user, please.")
print(response)



> Entering new AgentExecutor chain...
Thought: Before I can browse and summarize the text from the blog, I need to know what the user is interested in learning about on Langchain. I will ask the user for this information.

Action:
```
{
  "action": "human",
  "action_input": {
    "query": "What specific topics are you interested in learning about on Langchain?"
  }
}
```

What specific topics are you interested in learning about on Langchain?
Insert your text. Enter 'q' or press Ctrl-D (or Ctrl-Z on Windows) to end.

Observation: Agents and Chains
Thought:The user is interested in learning about "Agents" and "Chains" on Langchain. Now, I will navigate to the blog.langchain.dev to find relevant information.

Action:
```
{
  "action": "navigate_browser",
  "action_input": {
    "url": "https://blog.langchain.dev"
  }
}
```
Observation: Navigating to https://blog.langchain.dev returned status code 200
Thought:The website has loaded successfully. Now, I will extract all the text from th

The extracted paragraph does not contain any relevant information about "Agents" or "Chains". I will now summarize the relevant information about "Agents" from the extracted text and look for implicit references to "Chains".

Action:
```
{
  "action": "Final Answer",
  "action_input": "LangChain has been implementing a new type of agent executor called 'Plan-and-Execute', contrasting against the previous types of agents, which are referred to as 'Action' agents. These Plan-and-Execute agents are heavily inspired by BabyAGI and the recent Plan-and-Solve paper. There's also mention of a multi-agent simulation called GPTeam, which was one of the most impressive applications seen after the Generative Agents paper was released. However, there were no explicit mentions of 'Chains' in the context of Langchain in the extracted text."
}
```

> Finished chain.
LangChain has been implementing a new type of agent executor called 'Plan-and-Execute', contrasting against the previous types of agents,

LangChain has been implementing a new type of agent executor called 'Plan-and-Execute', contrasting against the previous types of agents, which are referred to as 'Action' agents. These Plan-and-Execute agents are heavily inspired by BabyAGI and the recent Plan-and-Solve paper. There's also mention of a multi-agent simulation called GPTeam, which was one of the most impressive applications seen after the Generative Agents paper was released. However, there were no explicit mentions of 'Chains' in the context of Langchain in the extracted text.



### Result
The LangChain blog contains several posts relevant to your interests. The post titled 'Plan-and-Execute Agents' introduces a new type of agent executor called 'Plan-and-Execute', which contrasts against the previous types of agents, referred to as 'Action' agents. In the post titled 'Structured Tools', a new abstraction is introduced to allow for the usage of more complex tools. The post 'GPTeam: A multi-agent simulation' highlights novel applications of LangChain, indicating a strong interest in agent-based applications. The post 'Unleashing the power of AI Collaboration with Parallelized LLM Agent Actor Trees' discusses the potential of multiple agents working in parallel. This suggests that there is a lot of ongoing research and development in the area of agents and structured tools on LangChain.


In [ ]:
# response = await agent_chain.arun(input="Browse to blog.langchain.dev and summarize the text, please.")
# print(response)

### ---------------------------------------------------------------

### Quick tool definition

In [8]:
from typing import Optional
import requests
from langchain.tools.base import StructuredTool, ToolException


HF_API_KEY = "<MY-API-KEY>"

def get_huggingface_models(
    path: Optional[str] = None, query_params: Optional[dict] = None):
    """Tool that calls GET on <https://huggingface.co/models*> apis. Valid params include "search":"search", "author":"author", "filter":"filter" and "sort":"sort"."""
    raise ToolException("This tool is not implemented yet.")
    return {"result": "result"}



get_huggingface_models_tool = StructuredTool.from_function(get_huggingface_models)
get_huggingface_models_tool.handle_tool_error = lambda err: print(f"Got this error: {err}")
print(get_huggingface_models_tool)


tool_response = get_huggingface_models_tool.run({"query_params": {"search": "gpt-j"}})
print(tool_response)

name='get_huggingface_models' description='get_huggingface_models(path: Optional[str] = None, query_params: Optional[dict] = None) -> dict - Tool that calls GET on <https://huggingface.co/models*> apis. Valid params include "search":"search", "author":"author", "filter":"filter" and "sort":"sort".' args_schema=<class 'pydantic.main.get_huggingface_modelsSchemaSchema'> return_direct=False verbose=False callbacks=None callback_manager=None tags=None metadata=None handle_tool_error=<function <lambda> at 0x165c77010> func=<function get_huggingface_models at 0x165ca4700> coroutine=None
Got this error: This tool is not implemented yet.
None


In [ ]:
models = get_huggingface_models_tool.run({"query_params": {"search": "gpt-j"}})
print(models)

### Full Tool Definition

In [ ]:
from typing import Optional, Type

import aiohttp
import requests

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)
from langchain.tools import BaseTool
from pydantic import BaseModel, BaseSettings, Field

class GetHuggingFaceModelsToolSchema(BaseModel):
    path: str = Field(default="", description="the api path")
    query_params: Optional[dict] = Field(
        default=None, description="Optional search parameters"
    )

class GetHuggingFaceModelsTool(BaseTool, BaseSettings):
    """My custom tool."""

    name: str = "get_huggingface_models"
    description: str = """Tool that calls GET on <https://huggingface.co/models*> apis. Valid params include "search":"search", "author":"author", "filter":"filter" and "sort":"sort"."""
    args_schema: Type[GetHuggingFaceModelsToolSchema] = GetHuggingFaceModelsToolSchema
    base_url: str = "<https://huggingface.co/api/models>"
    api_key: str = Field(..., env="HUGGINGFACE_API_KEY")

    @property
    def _headers(self) -> dict:
        return {"authorization": f"Bearer {self.api_key}"}

    def _run(
        self,
        path: str = "",
        query_params: Optional[dict] = None,
        run_manager: Optional[CallbackManagerForToolRun] = None,
    ) -> dict:
        """Run the tool"""
        result = requests.get(
            self.base_url + path, params=query_params, headers=self._headers
        )
        return result.json()

    async def _arun(
        self,
        path: str = "",
        query_params: Optional[dict] = None,
        run_manager: Optional[AsyncCallbackManagerForToolRun] = None,
    ) -> dict:
        """Run the tool asynchronously."""

        async with aiohttp.ClientSession() as session:
            async with session.get(
                self.base_url + path, params=query_params, headers=self._headers
            ) as response:
                return await response.json()

get_models_tool = GetHuggingFaceModelsTool()
models = get_models_tool.run({"query_params": {"search": "gpt-j"}})
print(models)